Checkin ne sert pas a grand chose pour notre taff, on l'enleve

In [1]:
import json
import os
from collections import Counter

In [2]:
city_counts = Counter()

# On ne lit que business.json pour choisir la ville
with open('data/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    for line in f:
        business = json.loads(line)
        city_counts[business['city']] += 1

# Affiche les 10 villes avec le plus de commerces
print("Top 10 des villes par nombre d'établissements :")
for city, count in city_counts.most_common(10):
    print(f"{city}: {count} commerces")

Top 10 des villes par nombre d'établissements :
Philadelphia: 14569 commerces
Tucson: 9250 commerces
Tampa: 9050 commerces
Indianapolis: 7540 commerces
Nashville: 6971 commerces
New Orleans: 6209 commerces
Reno: 5935 commerces
Edmonton: 5054 commerces
Saint Louis: 4827 commerces
Santa Barbara: 3829 commerces


on prend la ville avec le plus de commerce donc philly et on filtre l'ensemble des data

In [4]:

# Configuration
CITY_TARGET = "Philadelphia"
BUSINESS_SRC = 'data/yelp_academic_dataset_business.json'
REVIEWS_SRC = 'data/yelp_academic_dataset_review.json'
TIPS_SRC = 'data/yelp_academic_dataset_tip.json'
PHOTOS_SRC = 'data/photos.json'

# Extraire les restaurants de Philadelphie
philly_ids = set()

with open(BUSINESS_SRC, 'r', encoding='utf-8') as f_in, \
     open('philly_restaurants.json', 'w', encoding='utf-8') as f_out:

    for line in f_in:
        biz = json.loads(line)

        # Conditions :
        # 1) Ville = Philadelphia
        # 2) categories existe
        # 3) "Restaurants" dans categories
        if (
            biz.get('city') == CITY_TARGET
            and biz.get('categories') is not None
            and "Restaurants" in biz['categories']
        ):
            philly_ids.add(biz['business_id'])
            f_out.write(json.dumps(biz) + '\n')

print(f"Nombre de restaurants extraits : {len(philly_ids)}")

# Fonction pour filtrer les fichiers liés par business_id
def filter_related_file(input_file, output_file):
    count = 0
    with open(input_file, 'r', encoding='utf-8') as f_in, \
         open(output_file, 'w', encoding='utf-8') as f_out:

        for line in f_in:
            data = json.loads(line)
            if data.get('business_id') in philly_ids:
                f_out.write(json.dumps(data) + '\n')
                count += 1

    print(f"Terminé : {count} entrées enregistrées dans {output_file}.")

# Exécution du filtrage
filter_related_file(REVIEWS_SRC, 'philly_restaurant_reviews.json')
filter_related_file(TIPS_SRC, 'philly_restaurant_tips.json')
filter_related_file(PHOTOS_SRC, 'philly_restaurant_photos.json')


Nombre de restaurants extraits : 5852
Terminé : 687289 entrées enregistrées dans philly_restaurant_reviews.json.
Terminé : 87118 entrées enregistrées dans philly_restaurant_tips.json.
Terminé : 22295 entrées enregistrées dans philly_restaurant_photos.json.


On utilise le fichier des avis de Philadelphie pour collecter les user_id uniques

In [5]:

# Chemins des fichiers
reviews_philly = 'philly_restaurant_reviews.json'
users_source = 'data/yelp_academic_dataset_user.json'
users_philly = 'philly_restaurant_users.json'

# Collecter tous les user_id uniques présents dans les avis de Philadelphie
philly_user_ids = set()

with open(reviews_philly, 'r', encoding='utf-8') as f:
    for line in f:
        review = json.loads(line)
        # Le champ user_id est présent dans review.json 
        philly_user_ids.add(review['user_id'])


# Filtrer le fichier user original
count = 0

with open(users_source, 'r', encoding='utf-8') as f_in, \
     open(users_philly, 'w', encoding='utf-8') as f_out:
    for line in f_in:
        user_profile = json.loads(line)
        # On vérifie si cet utilisateur a écrit un avis à Philly 
        if user_profile.get('user_id') in philly_user_ids:
            f_out.write(json.dumps(user_profile) + '\n')
            count += 1

print(f"Terminé : {count} profils utilisateurs enregistrés dans {users_philly}.")

Terminé : 209513 profils utilisateurs enregistrés dans philly_restaurant_users.json.


On filtre maintenant le dossier photo

In [13]:

json_philly_photos = 'philly_restaurant_photos.json'
photos_folder = 'data/photos/' 

#Charger les IDs de photos que l'on veut garder
keep_photo_ids = set()
with open(json_philly_photos, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        keep_photo_ids.add(data['photo_id'])


for file_name in os.listdir(photos_folder):
    if file_name.endswith(".jpg"):
        # On enlève les photos qui ne sont pas dans le fichier
        photo_id = file_name.replace(".jpg", "")
        if photo_id not in keep_photo_ids:
            os.remove(os.path.join(photos_folder, file_name))
